**Hiển thị dữ liệu trong file**

In [1]:
import pandas as pd
from google.colab import drive
import numpy as np
drive.mount("/content/drive")
file_path = '/content/moso_filtered_20251024_164043.csv'

df = pd.read_csv(file_path)

print(df.head(10))
print(df.columns)
print(df.dtypes)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   index                                   address_original  \
0      1                3/16B Chánh Hưng, 4, 8, Hồ Chí Minh   
1      2          61/65 Đường Hùng Vương, 4, 5, Hồ Chí Minh   
2      3  77/48/1 Đường Lê Hoàng Phái, 17, Gò Vấp, Hồ Ch...   
3      4    440/13/302 Thống Nhất , 16, Gò Vấp, Hồ Chí Minh   
4      5  18A/111/14 Đường Nguyễn Thị Minh Khai, Đa Kao,...   
5      6  118/52 Đường Huỳnh Thiện Lộc, Hòa Thạnh, Tân P...   
6      7  436/59/72 Cách Mạng Tháng Tám, 11, 3, Hồ Chí Minh   
7      8                2A Lý Đạo Thành, 16, 8, Hồ Chí Minh   
8      9  31/7 Hoàng Văn Hợp, An Lạc A, Bình Tân, Hồ Chí...   
9     10  59/34A Đường Trương Đăng Quế, 3, Gò Vấp, Hồ Ch...   

                                             address                  price  \
0  3/16B Chánh Hưng, Phường 4, Quận 8, Thành phố ...             7600000000   
1  61/65 Đường Hùng

**Đưa các dữ liệu giá và kích thước về dạng số thực**

In [2]:
df['price'] = df['price'].astype(str).str.replace('.', '', regex=False)

df['price'] = pd.to_numeric(df['price'], errors='coerce').astype(np.float64)
df['pArea'] = pd.to_numeric(df['pArea'], errors='coerce').astype(np.float64)
df['pLandArea'] = pd.to_numeric(df['pLandArea'], errors='coerce').astype(np.float64)
print(df.dtypes)

index                   int64
address_original       object
address                object
price                 float64
type                   object
pType                  object
pWidth                float64
pLength               float64
pArea                 float64
pLandArea             float64
pNumberOfFloors       float64
pNumberOfBathrooms    float64
pNumberOfBedrooms     float64
pCertificateType       object
pFurnitureStatus       object
description            object
_createdAt             object
dtype: object


**Tính ngưỡng logarith trên và dưới để xử lý giá trị ngoại lai**

In [3]:
df['log_price'] = np.log(df['price']).astype(np.float64)

Q1 = df['log_price'].quantile(0.25)
Q3 = df['log_price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[
    (df['log_price'] < lower_bound) |
    (df['log_price'] > upper_bound)
]

print(f"Lower Logarithmic Bound: {lower_bound:.4f}")
print(f"Upper Logarithmic Bound: {upper_bound:.4f}")
print(f"Total ouliers found: {len(outliers)}")

outlier_data = outliers[['index', 'price', 'log_price', 'pArea', 'pLandArea', 'pType']].sort_values(by='log_price')

print(outlier_data)


Lower Logarithmic Bound: 20.7828
Upper Logarithmic Bound: 25.0204
Total ouliers found: 63
      index         price  log_price   pArea  pLandArea        pType
1870   1871  1.000000e+09  20.723266    19.1       19.1  alley_house
1988   1989  1.050000e+09  20.772056     0.0       15.0  alley_house
1458   1459  7.700000e+10  25.067071  1267.3      218.4    townhouse
1484   1485  7.900000e+10  25.092714   319.0      101.7    townhouse
1754   1755  8.000000e+10  25.105292     0.0      646.6    townhouse
...     ...           ...        ...     ...        ...          ...
1436   1437  4.100000e+16  38.252348    56.0       28.0  alley_house
515     516  4.110000e+16  38.254785    69.0        0.0        other
803     804  4.150000e+16  38.264470    64.0       39.0  alley_house
28       29  4.150000e+16  38.264470     0.0       29.0        other
219     220  4.150000e+16  38.264470    53.8       25.4        other

[63 rows x 6 columns]


**Xóa các giá trị ngoại lai**

In [4]:
indices_to_remove = outliers.index.tolist()
indices_to_keep = [1870, 1988]

for idx in indices_to_keep:
    try:
        indices_to_remove.remove(idx)
    except ValueError:
        pass

df = df.drop(indices_to_remove)
rows_actually_removed = len(indices_to_remove)
print(f"Số lượng hàng ngoại lai thực tế bị xóa khỏi DataFrame: {rows_actually_removed}")

Số lượng hàng ngoại lai thực tế bị xóa khỏi DataFrame: 61


 **Đếm số lượng giá trị số 0 hoặc NaN**

In [5]:
import pandas as pd
import numpy as np

columns_to_check = ['pWidth', 'pLength', 'pArea', 'pLandArea']
missing_or_zero_counts = {}

print(" Số lượng giá trị Bằng 0 HOẶC NaN trong mỗi cột ")

for col in columns_to_check:
    if col in df.columns:
        # 1. Đếm giá trị bằng 0
        zero_count = (df[col].eq(0)).sum()

        # 2. Đếm giá trị rỗng (NaN)
        nan_count = df[col].isna().sum()

        # 3. Tính tổng số lượng giá trị thiếu (0 HOẶC NaN)
        total_missing = zero_count + nan_count

        missing_or_zero_counts[col] = total_missing
        print(f"Cột '{col}': {total_missing} giá trị thiếu (bao gồm {zero_count} giá trị 0 và {nan_count} giá trị NaN)")

    else:
        print(f"Cột '{col}' không tồn tại trong DataFrame.")

 Số lượng giá trị Bằng 0 HOẶC NaN trong mỗi cột 
Cột 'pWidth': 80 giá trị thiếu (bao gồm 77 giá trị 0 và 3 giá trị NaN)
Cột 'pLength': 80 giá trị thiếu (bao gồm 77 giá trị 0 và 3 giá trị NaN)
Cột 'pArea': 374 giá trị thiếu (bao gồm 368 giá trị 0 và 6 giá trị NaN)
Cột 'pLandArea': 94 giá trị thiếu (bao gồm 80 giá trị 0 và 14 giá trị NaN)


**Xóa hàng có tất cả dữ liệu kích thước bằng 0 hoặc NaN**

In [6]:
import pandas as pd
import numpy as np

df_initial_rows = len(df)

core_area_cols = ['pArea', 'pLandArea']
core_size_cols = ['pWidth', 'pLength']
all_cols = core_area_cols + core_size_cols

def is_missing_or_zero(series):
    """Kiểm tra xem một giá trị là 0 HOẶC NaN"""
    return (series.eq(0)) | (series.isna())

condition_all_missing = is_missing_or_zero(df[all_cols]).all(axis=1)

final_removal_condition = condition_all_missing

indices_to_remove = df[final_removal_condition].index

df = df.drop(indices_to_remove)

rows_removed = len(indices_to_remove)
print(" Xóa hàng với các điều kiện thiếu dữ liệu (0 HOẶC NaN) nghiêm trọng ")
print(f"Số lượng hàng ban đầu: {df_initial_rows}")
print(f"Số lượng hàng đã xóa: {rows_removed}")
print(f"Số lượng dữ liệu còn lại: {len(df)}")

 Xóa hàng với các điều kiện thiếu dữ liệu (0 HOẶC NaN) nghiêm trọng 
Số lượng hàng ban đầu: 2084
Số lượng hàng đã xóa: 43
Số lượng dữ liệu còn lại: 2041


**Xử lý trường hợp pWidth và pLength bằng 0 hoặc bằng NaN**

In [7]:
import pandas as pd
import numpy as np

print(" Xử lý pWidth và pLength bằng 0 ")

median_pWidth = df[(df['pWidth'] > 0) & (df['pWidth'].notna())]['pWidth'].median()

print(f"Giá trị Trung vị (Median) của pWidth hợp lệ: {median_pWidth:.4f}")

pWidth_is_missing = (df['pWidth'] == 0) | (df['pWidth'].isna())

pLength_is_missing = (df['pLength'] == 0) | (df['pLength'].isna())

condition_impute= pWidth_is_missing & pLength_is_missing

indices_to_impute = df[condition_impute].index
rows_to_impute = len(indices_to_impute)

if rows_to_impute > 0:
    df.loc[indices_to_impute, 'pWidth'] = median_pWidth
    df.loc[indices_to_impute, 'pLength'] = np.where(
        df.loc[indices_to_impute, 'pWidth'] != 0,
        df.loc[indices_to_impute, 'pArea'] / df.loc[indices_to_impute, 'pWidth'], 0
    )

    print(f"Đã xử lý và lấp đầy {rows_to_impute} hàng thỏa mãn điều kiện.")

else:
    print("Không tìm thấy hàng nào mà pWidth và pLength đều bằng 0 để xử lý.")

print("\nKiểm tra lại số lượng giá trị 0 sau khi xử lý:")
print(f"pWidth còn lại: {(df['pWidth'].eq(0)).sum()} giá trị 0")
print(f"pLength còn lại: {(df['pLength'].eq(0)).sum()} giá trị 0")

 Xử lý pWidth và pLength bằng 0 
Giá trị Trung vị (Median) của pWidth hợp lệ: 4.0500
Đã xử lý và lấp đầy 37 hàng thỏa mãn điều kiện.

Kiểm tra lại số lượng giá trị 0 sau khi xử lý:
pWidth còn lại: 0 giá trị 0
pLength còn lại: 0 giá trị 0


**Gán giá trị trong trường hợp pArea hoặc pLandArea là NaN hoặc bằng 0**

In [8]:
import numpy as np
import pandas as pd

print(" Xử lý giá trị thiếu (0 HOẶC NaN) bằng cách lấp đầy chéo ")

#  Xử lý 1: Lấp đầy pLandArea khi nó thiếu và pArea hợp lệ
pLandArea_is_missing = (df['pLandArea'] == 0) | (df['pLandArea'].isna())

pArea_is_valid = (df['pArea'] != 0) & (df['pArea'].notna())

condition_1 = pLandArea_is_missing & pArea_is_valid

df['pLandArea'] = np.where(
    condition_1,
    df['pArea'],
    df['pLandArea']
)
rows_filled_pLandArea = condition_1.sum()
print(f"Đã lấp đầy {rows_filled_pLandArea} giá trị thiếu/0 trong cột 'pLandArea' bằng giá trị 'pArea'.")

#  Xử lý 2: Lấp đầy pArea khi nó thiếu và pLandArea hợp lệ
pArea_is_missing = (df['pArea'] == 0) | (df['pArea'].isna())

pLandArea_is_valid = (df['pLandArea'] != 0) & (df['pLandArea'].notna())

condition_2 = pArea_is_missing & pLandArea_is_valid

df['pArea'] = np.where(
    condition_2,
    df['pLandArea'],
    df['pArea']
)
rows_filled_pArea = condition_2.sum()
print(f"Đã lấp đầy {rows_filled_pArea} giá trị thiếu/0 trong cột 'pArea' bằng giá trị 'pLandArea'.")

print("\nKiểm tra số lượng giá trị thiếu còn lại:")

# Kiểm tra số lượng giá trị thiếu (0 HOẶC NaN) còn lại
pLandArea_missing_count = (df['pLandArea'] == 0).sum() + df['pLandArea'].isna().sum()
pArea_missing_count = (df['pArea'] == 0).sum() + df['pArea'].isna().sum()

print(f"pLandArea còn lại: {pLandArea_missing_count} giá trị thiếu (0 hoặc NaN)")
print(f"pArea còn lại: {pArea_missing_count} giá trị thiếu (0 hoặc NaN)")

 Xử lý giá trị thiếu (0 HOẶC NaN) bằng cách lấp đầy chéo 
Đã lấp đầy 45 giá trị thiếu/0 trong cột 'pLandArea' bằng giá trị 'pArea'.
Đã lấp đầy 325 giá trị thiếu/0 trong cột 'pArea' bằng giá trị 'pLandArea'.

Kiểm tra số lượng giá trị thiếu còn lại:
pLandArea còn lại: 6 giá trị thiếu (0 hoặc NaN)
pArea còn lại: 6 giá trị thiếu (0 hoặc NaN)


**Thay thế các giá trị thiếu của pArea và pLandArea bằng pWidth và pLength**

In [9]:
import numpy as np
import pandas as pd

print(" Lấp đầy diện tích bị thiếu (0 HOẶC NaN) bằng Kích thước (pWidth * pLength) ")

is_area_missing = (df['pArea'].eq(0)) | (df['pArea'].isna())
is_land_area_missing = (df['pLandArea'].eq(0)) | (df['pLandArea'].isna())
is_width_valid = (df['pWidth'] > 0) & (df['pWidth'].notna())
is_length_valid = (df['pLength'] > 0) & (df['pLength'].notna())

estimated_area = df['pWidth'] * df['pLength']

final_imputation_condition = (
    is_area_missing &
    is_land_area_missing &
    is_width_valid &
    is_length_valid
)

df['pLandArea'] = np.where(
    final_imputation_condition,
    estimated_area,
    df['pLandArea']
)
rows_filled_pLandArea = final_imputation_condition.sum()

df['pArea'] = np.where(
    final_imputation_condition,
    estimated_area,
    df['pArea']
)
rows_filled_pArea = final_imputation_condition.sum()


print(f"Đã lấp đầy {rows_filled_pLandArea} giá trị thiếu/0 trong cột 'pLandArea'.")
print(f"Đã lấp đầy {rows_filled_pArea} giá trị thiếu/0 trong cột 'pArea'.")

print("\nKiểm tra số lượng giá trị thiếu (0 hoặc NaN) còn lại:")
pLandArea_missing = (df['pLandArea'].eq(0)).sum() + df['pLandArea'].isna().sum()
pArea_missing = (df['pArea'].eq(0)).sum() + df['pArea'].isna().sum()

print(f"pLandArea còn lại: {pLandArea_missing} giá trị thiếu.")
print(f"pArea còn lại: {pArea_missing} giá trị thiếu.")

 Lấp đầy diện tích bị thiếu (0 HOẶC NaN) bằng Kích thước (pWidth * pLength) 
Đã lấp đầy 6 giá trị thiếu/0 trong cột 'pLandArea'.
Đã lấp đầy 6 giá trị thiếu/0 trong cột 'pArea'.

Kiểm tra số lượng giá trị thiếu (0 hoặc NaN) còn lại:
pLandArea còn lại: 0 giá trị thiếu.
pArea còn lại: 0 giá trị thiếu.


**Kiểm tra lại xem còn giá trị bị thiếu không**

In [10]:
import pandas as pd
import numpy as np

columns_to_check = ['pWidth', 'pLength', 'pArea', 'pLandArea']
missing_or_zero_counts = {}

print(" Số lượng giá trị Bằng 0 HOẶC NaN trong mỗi cột ")

for col in columns_to_check:
    if col in df.columns:
        # 1. Đếm giá trị bằng 0
        zero_count = (df[col].eq(0)).sum()

        # 2. Đếm giá trị rỗng (NaN)
        nan_count = df[col].isna().sum()

        # 3. Tính tổng số lượng giá trị thiếu (0 HOẶC NaN)
        total_missing = zero_count + nan_count

        missing_or_zero_counts[col] = total_missing
        print(f"Cột '{col}': {total_missing} giá trị thiếu (bao gồm {zero_count} giá trị 0 và {nan_count} giá trị NaN)")

    else:
        print(f"Cột '{col}' không tồn tại trong DataFrame.")

 Số lượng giá trị Bằng 0 HOẶC NaN trong mỗi cột 
Cột 'pWidth': 0 giá trị thiếu (bao gồm 0 giá trị 0 và 0 giá trị NaN)
Cột 'pLength': 0 giá trị thiếu (bao gồm 0 giá trị 0 và 0 giá trị NaN)
Cột 'pArea': 0 giá trị thiếu (bao gồm 0 giá trị 0 và 0 giá trị NaN)
Cột 'pLandArea': 0 giá trị thiếu (bao gồm 0 giá trị 0 và 0 giá trị NaN)


**Kiểm tra các giá trị trong cột pArea và pLandArea**

In [11]:
import pandas as pd

columns_to_show = ['pArea', 'pLandArea']

for col in columns_to_show:
    if col in df.columns:
        print(f" 10 Giá trị lớn nhất trong cột '{col}' ")
        sorted_values = df[col].dropna().sort_values(ascending=False).head(10)
        print(sorted_values)
        print('\n')
    else:
        print(f": Cột '{col}' không tồn tại trong DataFrame.")

 10 Giá trị lớn nhất trong cột 'pArea' 
1088    7832.0
1281    1365.0
349     1209.0
406     1066.4
1885    1038.2
500     1009.0
1791     989.2
469      983.7
1390     935.8
978      933.8
Name: pArea, dtype: float64


 10 Giá trị lớn nhất trong cột 'pLandArea' 
1088    7832.00
1162    2423.00
1924    1266.10
1885    1038.20
978      933.80
1876     705.00
41       637.80
1887     628.25
783      616.90
301      606.00
Name: pLandArea, dtype: float64




**Xóa 2 giá trị có lỗi nhập liệu sai khác lớn**

In [12]:
import pandas as pd

col_to_sort = 'pLandArea'

df_sorted = df.sort_values(by=col_to_sort, ascending=False)

indices_to_drop = df_sorted.head(2).index.tolist()

initial_rows = len(df)

df.drop(indices_to_drop, inplace=True)

print(f" Xóa Outliers Lớn nhất ")
print(f"Đã xóa thành công 2 hàng có giá trị '{col_to_sort}' lớn nhất.")
print(f"Các index đã bị xóa là: {indices_to_drop}")
print(f"Số lượng hàng ban đầu: {initial_rows}")
print(f"Số lượng hàng còn lại: {len(df)}")

 Xóa Outliers Lớn nhất 
Đã xóa thành công 2 hàng có giá trị 'pLandArea' lớn nhất.
Các index đã bị xóa là: [1088, 1162]
Số lượng hàng ban đầu: 2041
Số lượng hàng còn lại: 2039


**Xử lý các giá trị NaN**

In [13]:
import pandas as pd
import numpy as np

df_initial_rows = len(df)

cols_numeric = [
    'pNumberOfFloors', 'pNumberOfBathrooms', 'pNumberOfBedrooms'
]
col_categorical = ['pFurnitureStatus', 'pCertificateType']
all_cols_to_check = cols_numeric + col_categorical

print(" Lấp đầy Giá trị Rỗng (NaN) ")

for col in cols_numeric:
    if col in df.columns:
        median_value = df[col].median()
        rows_filled = df[col].isna().sum()

        df[col] = df[col].fillna(median_value)

        print(f"Cột '{col}': Đã lấp đầy {rows_filled} giá trị NaN bằng Trung vị ({median_value:.2f}).")
    else:
        print(f": Cột số '{col}' không tồn tại.")

for col_categorical_index in col_categorical:
  if col_categorical_index in df.columns:
      rows_filled_cat = df[col_categorical_index].isna().sum()
      df[col_categorical_index] = df[col_categorical_index].fillna('other')

      print(f"Cột '{col_categorical_index}': Đã lấp đầy {rows_filled_cat} giá trị NaN bằng 'other'.")
  else:
      print(f": Cột phân loại '{col_categorical_index}' không tồn tại.")

print("\n Kiểm tra lại số lượng giá trị rỗng (NaN) ")
total_nan_after_imputation = 0
for col in all_cols_to_check:
    if col in df.columns:
        nan_count_after = df[col].isna().sum()
        total_nan_after_imputation += nan_count_after
        print(f"Cột '{col}': {nan_count_after} giá trị NaN còn lại.")

print(f"\nTổng số NaN còn lại trong các cột đã xử lý: {total_nan_after_imputation}")

 Lấp đầy Giá trị Rỗng (NaN) 
Cột 'pNumberOfFloors': Đã lấp đầy 6 giá trị NaN bằng Trung vị (3.00).
Cột 'pNumberOfBathrooms': Đã lấp đầy 4 giá trị NaN bằng Trung vị (3.00).
Cột 'pNumberOfBedrooms': Đã lấp đầy 4 giá trị NaN bằng Trung vị (3.00).
Cột 'pFurnitureStatus': Đã lấp đầy 9 giá trị NaN bằng 'other'.
Cột 'pCertificateType': Đã lấp đầy 0 giá trị NaN bằng 'other'.

 Kiểm tra lại số lượng giá trị rỗng (NaN) 
Cột 'pNumberOfFloors': 0 giá trị NaN còn lại.
Cột 'pNumberOfBathrooms': 0 giá trị NaN còn lại.
Cột 'pNumberOfBedrooms': 0 giá trị NaN còn lại.
Cột 'pFurnitureStatus': 0 giá trị NaN còn lại.
Cột 'pCertificateType': 0 giá trị NaN còn lại.

Tổng số NaN còn lại trong các cột đã xử lý: 0


**Tách các quận thuộc địa chỉ**

In [14]:
columns_to_drop1 = ['index', 'address_original', 'type', 'description', '_createdAt']
df = df.drop(columns=columns_to_drop1, errors='ignore')

df['address_part'] = df['address'].str.split(',')
df['district'] = df['address_part'].str[-2].str.strip()
df = df.drop(columns=['address_part', 'address'], errors='ignore')

print("\n5 giá trị đầu tiên của cột 'district' mới:")
print(df['district'].head())


5 giá trị đầu tiên của cột 'district' mới:
0    Quận 8
1    Quận 5
2    Gò Vấp
3    Gò Vấp
4    Quận 1
Name: district, dtype: object


**Chuẩn hóa cho tên quận**

In [15]:
import pandas as pd
import re

district_col = 'district'

def normalize_district_name(district):

    # Chuẩn hóa tên quận

    if pd.isna(district):
        return district

    district = str(district).strip()

    district_lower = district.lower()

    if 'thủ đức cũ' in district_lower:
        return 'Quận Thủ Đức'

    if re.match(r'^\d+$', district):
        return 'Quận ' + district

    # Kiểm tra xem chuỗi đã bắt đầu bằng 'Quận ' hoặc 'Huyện ' hay chưa
    if not (district_lower.startswith('quận ') or district_lower.startswith('huyện ')):
        return 'Quận ' + district
    return district

# Áp dụng hàm chuẩn hóa cho toàn bộ cột 'district'
df[district_col] = df[district_col].apply(normalize_district_name)

print("Đã thêm tiền tố 'Quận ' cho tất cả các hàng thiếu.")
print(df[district_col].unique()[:10])

Đã thêm tiền tố 'Quận ' cho tất cả các hàng thiếu.
['Quận 8' 'Quận 5' 'Quận Gò Vấp' 'Quận 1' 'Quận Tân Phú' 'Quận 3'
 'Quận Bình Tân' 'Quận 12' 'Quận Tân Bình' 'Quận 10']


**Tải file về máy dùng cho visualize**

In [16]:
import pandas as pd
from google.colab import files

df_cleaned = df.copy()

file_name = 'data_for_visualize.csv'

df_cleaned.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"Đã lưu DataFrame thành công dưới tên '{file_name}'")
files.download(file_name)

Đã lưu DataFrame thành công dưới tên 'data_for_visualize.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Thực hiện one-hot encoding**

In [17]:
import pandas as pd
import numpy as np

col_district = 'district'
cols_with_is_prefix = ['pType', 'pCertificateType', 'pFurnitureStatus']

print(" Thực hiện One-Hot Encoding Tùy chỉnh ")

district_dummies = pd.get_dummies(df[col_district], drop_first=True)

other_dummies = pd.get_dummies(df[cols_with_is_prefix], drop_first=True)

df_cleaned = df.drop(columns=[col_district] + cols_with_is_prefix, errors='ignore').copy()

df_cleaned = pd.concat([df_cleaned, district_dummies, other_dummies], axis=1)

#Điều chỉnh tên cột
new_columns = df_cleaned.columns

# Duyệt và đổi tên
rename_map = {}
for col in new_columns:
    #Thêm tiền tố 'is_' cho các cột pType, pCertificateType, pFurnitureStatus
    for prefix in cols_with_is_prefix:
        if col.startswith(prefix + '_'):
            # Xử lý tên cột nếu nó là danh mục 'other'
            if col.endswith('_other'):
                new_name = f"{prefix}_is_other"
            else:
                # Thêm tiền tố 'is_' và xóa tiền tố gốc
                new_name = f"is_{col[len(prefix) + 1:]}"
            rename_map[col] = new_name
            break # Chuyển sang cột tiếp theo

df_cleaned.rename(columns=rename_map, inplace=True)

print("Hoàn thành mã hóa. Các cột mới đã được tạo và đổi tên.")
print("\nKiểm tra các cột OHE (ví dụ):")
print([col for col in df_cleaned.columns if 'is_' in col or col in district_dummies.columns][:10])

df = df_cleaned.copy()

 Thực hiện One-Hot Encoding Tùy chỉnh 
Hoàn thành mã hóa. Các cột mới đã được tạo và đổi tên.

Kiểm tra các cột OHE (ví dụ):
['Quận 1', 'Quận 10', 'Quận 11', 'Quận 12', 'Quận 2', 'Quận 3', 'Quận 4', 'Quận 5', 'Quận 6', 'Quận 7']


**Chuyển true/false về 1/0**

In [18]:
# Lấy danh sách các cột OHE (có kiểu dữ liệu là bool)
ohe_cols = df.select_dtypes(include=['bool']).columns

# Thực hiện chuyển đổi
df[ohe_cols] = df[ohe_cols].astype(int)

print("Đã chuyển đổi các cột One-Hot từ True/False sang 1/0 thành công.")
print(df[ohe_cols].head(3))

Đã chuyển đổi các cột One-Hot từ True/False sang 1/0 thành công.
   Quận 1  Quận 10  Quận 11  Quận 12  Quận 2  Quận 3  Quận 4  Quận 5  Quận 6  \
0       0        0        0        0       0       0       0       0       0   
1       0        0        0        0       0       0       0       1       0   
2       0        0        0        0       0       0       0       0       0   

   Quận 7  ...  is_townhouse  is_villa  pCertificateType_is_other  \
0       0  ...             0         0                          0   
1       0  ...             0         0                          0   
2       0  ...             0         0                          0   

   is_pending_certificate  is_share_certificate  is_transfer_agreement  \
0                       0                     0                      0   
1                       0                     0                      0   
2                       0                     0                      0   

   is_under_completion  pFurnitureStatus

Kiểm tra số lượng giá trị là 1 trong các cột one-hot

In [19]:
import pandas as pd

ohe_prefixes = [
    'Quận', 'Huyện', 'is_', 'pType_', 'pCertificateType_', 'pFurnitureStatus_'
]

# 1. Lọc ra các cột OHE
ohe_cols_names = [
    col for col in df.columns if
    col.startswith('Quận') or
    col.startswith('Huyện') or # Thêm Huyện nếu cần
    col.startswith('is_') or
    col.startswith('pType_') or
    col.startswith('pCertificateType_') or
    col.startswith('pFurnitureStatus_')
]

print("--- Tần suất Xuất hiện của các Danh mục OHE (Số lượng giá trị '1') ---")

ohe_counts = df[ohe_cols_names].sum().sort_values(ascending=False)

print("\n6 Cột One-Hot có Tần suất cao nhất (Nhiều '1' nhất):")
print(ohe_counts.head(10))

print("\n" + "="*50)

print("6 Cột One-Hot có Tần suất thấp nhất (Ít '1' nhất):")
print(ohe_counts.tail(10))

--- Tần suất Xuất hiện của các Danh mục OHE (Số lượng giá trị '1') ---

6 Cột One-Hot có Tần suất cao nhất (Nhiều '1' nhất):
pFurnitureStatus_is_other    1323
is_alley_house                959
is_partially_furnished        596
pType_is_other                569
is_townhouse                  330
Quận Bình Tân                 236
Quận Tân Bình                 235
Quận Gò Vấp                   231
Quận Tân Phú                  198
Quận 12                       155
dtype: int64

6 Cột One-Hot có Tần suất thấp nhất (Ít '1' nhất):
Quận 2                       15
is_shophouse                 15
Quận Bình Chánh              14
Quận Củ Chi                   7
is_share_certificate          5
pCertificateType_is_other     4
is_apartment_building         2
is_pending_certificate        1
is_transfer_agreement         1
is_under_completion           1
dtype: int64


**Gộp các cột one-hot**

In [20]:
import pandas as pd
import numpy as np

certificate_cols_to_merge = [
    'is_share_certificate',
    'is_under_completion',
    'is_pending_certificate',
    'is_transfer_agreement'
]

existing_certificate_cols = [col for col in certificate_cols_to_merge if col in df.columns]

if existing_certificate_cols:
    if 'pCertificateType_is_other' not in df.columns:
        df['pCertificateType_is_other'] = 0

    df['pCertificateType_is_other'] = df[existing_certificate_cols + ['pCertificateType_is_other']].max(axis=1)

    df.drop(columns=existing_certificate_cols, inplace=True)

    print(f"Đã gộp và xóa các cột: {existing_certificate_cols}")
    print(f"Giá trị mới của 'pCertificateType_is_other': {df['pCertificateType_is_other'].sum()} (Số lượng '1')")
else:
    print("Không tìm thấy các cột cần gộp trong DataFrame.")


#Gộp cột is_apartment_building vào is_apartment
col_source = 'is_apartment_building'
col_target = 'is_apartment'

if col_source in df.columns and col_target in df.columns:
    df[col_target] = np.maximum(df[col_target], df[col_source])

    df.drop(columns=[col_source], inplace=True)

    print(f"Giá trị mới của '{col_target}': {df[col_target].sum()} (Số lượng '1')")
elif col_source in df.columns or col_target in df.columns:
    print("Một hoặc cả hai cột 'is_apartment_building'/'is_apartment' không tồn tại.")

Đã gộp và xóa các cột: ['is_share_certificate', 'is_under_completion', 'is_pending_certificate', 'is_transfer_agreement']
Giá trị mới của 'pCertificateType_is_other': 12 (Số lượng '1')
Giá trị mới của 'is_apartment': 38 (Số lượng '1')


**Giảm chiều dữ liệu sử dụng PCA**

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
cols_for_pca = ['pWidth', 'pLength', 'pArea', 'pLandArea']

# Khởi tạo Scaler và chuẩn hóa dữ liệu
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[cols_for_pca])

pca = PCA(n_components=3)

principal_components = pca.fit_transform(df_scaled)

df_pca = pd.DataFrame(data = principal_components,
                     columns = ['PC1_Size', 'PC2_Size', 'PC3_Size'],
                     index = df.index)

# Kiểm tra tỷ lệ phương sai được giải thích
explained_variance_ratio = pca.explained_variance_ratio_.sum()
print(" Kết quả Phân tích Thành phần Chính (PCA) ")
print(f"Tổng phương sai được giải thích bởi 3 PC: {explained_variance_ratio*100:.2f}%")

df.drop(columns=cols_for_pca, inplace=True)

df_final = pd.concat([df, df_pca], axis=1)

print(f"\nDataFrame cuối cùng đã được giảm chiều. Số cột mới là:")
print(df_final[['PC1_Size', 'PC2_Size', 'PC3_Size']].head())

# Gán lại df
df = df_final.copy()

 Kết quả Phân tích Thành phần Chính (PCA) 
Tổng phương sai được giải thích bởi 3 PC: 94.13%

DataFrame cuối cùng đã được giảm chiều. Số cột mới là:
   PC1_Size  PC2_Size  PC3_Size
0 -0.457528 -0.409690 -0.344591
1 -1.067163 -0.042720 -0.103576
2 -0.691957  0.477146 -0.058091
3 -1.268494  0.061789  0.040619
4 -1.101780 -0.249518  0.074421


**Tải file training**

In [22]:
import pandas as pd
from google.colab import files

df_cleaned = df.copy()

if 'price' in df_cleaned.columns:
    df_cleaned.drop(columns=['price'], inplace=True)

cols_to_front = ['PC1_Size', 'PC2_Size', "PC3_Size"]

current_cols = df_cleaned.columns.tolist()

other_cols = [col for col in current_cols if col not in cols_to_front]

new_col_order = cols_to_front + other_cols

df_cleaned = df_cleaned[new_col_order]

file_name = 'data_for_training.csv'

df_cleaned.to_csv(file_name, index=False, encoding='utf-8-sig')

print(f"Đã lưu DataFrame thành công dưới tên '{file_name}'")

files.download(file_name)

Đã lưu DataFrame thành công dưới tên 'data_for_training.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>